# Config

In [1]:
import ConnectionConfig as cc
cc.setupEnvironment()

# Start local cluster

In [7]:
spark = cc.startLocalCluster("DIM_CUSTOMER")
spark.getActiveSession()

In [3]:
#Extract
cc.set_connectionProfile("velodb")

df_user = spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "velo_users") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "userid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 60000) \
    .load()

df_user.show(5)




+------+-----------------+--------------------+--------------------+------+-------+-------------------+------------+
|userid|             name|               email|              street|number|zipcode|               city|country_code|
+------+-----------------+--------------------+--------------------+------+-------+-------------------+------------+
|     1|      Bouman Lars|Lars.Bouman@gmail...|         Somméstraat|  156 |   2060|          Antwerpen|          BE|
|     2|van der Zee Julia|Julia.van.der.Zee...|          Europalaan|   43 |   2610|Wilrijk (Antwerpen)|          BE|
|     3|  de Boer Ricardo|Ricardo.de.Boer@g...|   Maria Clarastraat|   80 |   2160|          Wommelgem|          BE|
|     4|   Willems Angela|Angela.Willems@te...|Graaf Joseph de P...|   15 |   2900|            Schoten|          BE|
|     5| Heijnen Patricia|Patricia.Heijnen@...|          Meylstraat|  111 |   2540|               Hove|          BE|
+------+-----------------+--------------------+-----------------

In [5]:
#Transform
df_user.createOrReplaceTempView("source_User")
#df_dim_user= spark.sql("select * from source_User")
#df_dim_user.show(5)

df_dim_user= spark.sql("select uuid() as userSK,*,to_timestamp('1900-01-01','yyyy-MM-dd') as scd_start,to_timestamp('2100-12-12','yyyy-MM-dd') as scd_end,True as current,md5(concat(name, country_code, city, street, number)) as md5 from source_User")
df_dim_user.show(5)

#df_user = spark.sql("select uuid() as userSK,*, to_timestamp('1999-01-01','yyyy-MM-dd') as scd_start, to_timestamp('2100-12-12','yyyy-MM-dd') as scd_end,md5(concat( name, city, street, number)) as md5, True as current from dimUser")

#df_user.printSchema()
#df_user.show()

+--------------------+------+-----------------+--------------------+--------------------+------+-------+-------------------+------------+-------------------+-------------------+-------+--------------------+
|              userSK|userid|             name|               email|              street|number|zipcode|               city|country_code|          scd_start|            scd_end|current|                 md5|
+--------------------+------+-----------------+--------------------+--------------------+------+-------+-------------------+------------+-------------------+-------------------+-------+--------------------+
|bb34a292-647e-47d...|     1|      Bouman Lars|Lars.Bouman@gmail...|         Somméstraat|  156 |   2060|          Antwerpen|          BE|1900-01-01 00:00:00|2100-12-12 00:00:00|   true|b9d7c0d8b9977d6cb...|
|49b7d470-35ad-404...|     2|van der Zee Julia|Julia.van.der.Zee...|          Europalaan|   43 |   2610|Wilrijk (Antwerpen)|          BE|1900-01-01 00:00:00|2100-12-12 00:0

In [6]:
#Load
df_dim_user.write.format("delta").mode("overwrite").saveAsTable("dimUser")

In [8]:
spark.stop()

# Klad hieronder

In [1]:
from pyspark.sql import SparkSession

def startSimpleCluster(appName):
    spark = SparkSession.builder \
        .appName(appName) \
        .config("spark.master", "local[1]") \
        .config("spark.driver.host", "127.0.0.1") \
        .getOrCreate()
    return spark

if __name__ == "__main__":
    spark = startSimpleCluster("TestApp")
    print("Spark session created successfully!")
    spark.stop()

Spark session created successfully!


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Check Temp Dir") \
    .master("local[*]") \
    .getOrCreate()

print("Temporary directory Spark is using:")
print(spark.sparkContext.getConf().get("spark.local.dir"))

spark.stop()

Temporary directory Spark is using:
None


In [30]:
users_df = spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "velo_users") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "userid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 20) \
    .load()

subscriptions_df = spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "Subscriptions") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .load()

subscription_types_df = spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "Subscription_types") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .load()




In [31]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lead

# Voeg de data samen
combined_df = subscriptions_df.join(users_df, "UserId") \
    .join(subscription_types_df, "SubscriptionTypeId")

# Bereken de einddatum met een window-functie
window_spec = Window.partitionBy("UserId").orderBy("ValidFrom")
combined_df = combined_df.withColumn("end_date", lead("ValidFrom", 1).over(window_spec))


In [32]:
from pyspark.sql.functions import lit, when

scd_df = combined_df.withColumn("is_current", when(col("end_date").isNull(), lit(True)).otherwise(lit(False)))


In [33]:
scd_df.write.format("delta").mode("overwrite").save("/path/to/customer_dimension")


In [36]:
from pyspark.sql.functions import current_date, lit

# Lees bestaande klantdimensie in
existing_data = spark.read.format("delta").load("/path/to/customer_dimension")

# Lees nieuwe gegevens in (vervang 'jdbc-url' en tabellen met jouw configuratie)
new_data =   spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "velo_users") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "userid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 20) \
    .load()

subscriptions = spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "Subscriptions") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .load()

subscription_types = spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "Subscription_types") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .load()

# Combineer de tabellen
new_data = new_data.join(subscriptions, "UserId") \
    .join(subscription_types, "SubscriptionTypeId")

# Controleer de kolomnamen
print("Kolommen in nieuwe data:")
new_data.printSchema()

print("Kolommen in bestaande dimensie:")
existing_data.printSchema()

# Gebruik de juiste kolom voor de join (bijv. `UserId`)
updates = new_data.join(existing_data, "UserId", "left_anti")  # Alleen nieuwe of gewijzigde data

# Update bestaande rijen
updated_existing = existing_data.join(updates, "UserId") \
    .withColumn("end_date", current_date()) \
    .withColumn("is_current", lit(False))

# Voeg nieuwe rijen in
new_rows = updates.withColumn("start_date", current_date()) \
    .withColumn("end_date", lit(None).cast("date")) \
    .withColumn("is_current", lit(True))

# Combineer en sla op
final_data = updated_existing.union(new_rows)
final_data.write.format("delta").mode("overwrite").save("/path/to/customer_dimension")

Kolommen in nieuwe data:
root
 |-- subscriptiontypeid: integer (nullable = true)
 |-- userid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- street: string (nullable = true)
 |-- number: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- subscriptionid: integer (nullable = true)
 |-- validfrom: date (nullable = true)
 |-- description: string (nullable = true)

Kolommen in bestaande dimensie:
root
 |-- subscriptiontypeid: integer (nullable = true)
 |-- userid: integer (nullable = true)
 |-- subscriptionid: integer (nullable = true)
 |-- validfrom: date (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- street: string (nullable = true)
 |-- number: string (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country_code: string (nullable = true)
 

AnalysisException: [NUM_COLUMNS_MISMATCH] UNION can only be performed on inputs with the same number of columns, but the first input has 25 columns and the second input has 15 columns.;
'Union false, false
:- Project [userid#1396, subscriptiontypeid#1395, subscriptionid#1397, validfrom#1398, name#1399, email#1400, street#1401, number#1402, zipcode#1403, city#1404, country_code#1405, description#1406, end_date#1527, false AS is_current#1553, subscriptiontypeid#1441, name#1424, email#1425, street#1426, number#1427, zipcode#1428, city#1429, country_code#1430, subscriptionid#1439, validfrom#1440, description#1448]
:  +- Project [userid#1396, subscriptiontypeid#1395, subscriptionid#1397, validfrom#1398, name#1399, email#1400, street#1401, number#1402, zipcode#1403, city#1404, country_code#1405, description#1406, current_date(Some(Europe/Brussels)) AS end_date#1527, is_current#1408, subscriptiontypeid#1441, name#1424, email#1425, street#1426, number#1427, zipcode#1428, city#1429, country_cod